In [13]:
# import some necessary functions for plotting as well as the diffusion_map class from pydiffmap.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import pyplot
from keras.datasets import mnist
from mpl_toolkits.mplot3d import Axes3D
from pydiffmap import diffusion_map as dm
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import eigh
import plotly.graph_objs as go
from plotly.subplots import make_subplots

%matplotlib inline

## Set up

In [196]:
def make_plot(N=1000, rseed=42):
    # Make a plot with "HELLO" text; save as PNG
    fig, ax = plt.subplots(figsize=(4, 1))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ax.axis('off')
    ax.text(0.5, 0.4, 'T', va='center', ha='center', size=85)
    fig.savefig('plot.png')
    plt.close(fig)
    fig.show()
    # Open this PNG and draw random points from it
    from matplotlib.image import imread
    data = imread('plot.png')[::-1, :, 0].T
    rng = np.random.RandomState(rseed)
    X = rng.rand(5 * N, 2)
    i, j = (X * data.shape).astype(int).T
    mask = (data[i, j] < 1)
    X = X[mask]
    X[:, 0] *= (data.shape[0] / data.shape[1])
    X = X[:N]
    return X[np.argsort(X[:, 0])]

In [197]:
# Generate shape
X = make_plot(3000)
data=go.Scatter(x=X[:, 0], y=X[:, 1], mode='markers', marker=dict(
        size=3,color=X[:,1],opacity=1,colorscale='Viridis'))
layout=go.Layout(title_text="Original shape", title_x=0.5, title_y=0.8,title_font_size=12)
fig = go.Figure(data=[data], layout=layout)
fig.update_layout(height=400, width=400,showlegend=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

C:\Users\tibyr\AppData\Local\Temp/ipykernel_32044/3457705633.py:9: UserWarning:

Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.



In [198]:
# Adding new dimension to the original geometric structure
newX = pd.DataFrame(X)
newX['dimension1'] = [random.uniform(0.1,0.5) for _ in range(len(X))]
newX = np.asarray(newX)

In [199]:
def plot_3Dfigure(newX, title='Datapoints'):
    data = go.Scatter3d(x=newX[:,0], y=newX[:,1], z=newX[:,2], mode='markers', marker=dict(
            size=2,color=newX[:,1],opacity=0.7,colorscale='Viridis'))
    layout = go.Layout(title_text=title,title_x=0.5,title_y=0.8,title_font_size=12)
    fig = go.Figure(data=[data], layout=layout)
    fig.update_layout(showlegend=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.update_layout(scene = dict(
                    xaxis = dict(title= '', ticks= '', showticklabels= False,),
                    yaxis = dict(title= '', ticks= '', showticklabels= False,),
                    zaxis = dict(title= '', ticks= '', showticklabels= False,),
                    ))
                  
    fig.show()

In [200]:
plot_3Dfigure(newX, title='Synthetic 3D Datapoints')

## Diffusion Maps

In [214]:
def plot_2Dsub_figures(d_map, title='Diffused points'):
    data=go.Scatter(x=d_map[:,0], y=d_map[:,1], mode='markers', marker=dict(
        size=3,color=d_map[:,1],opacity=0.8,colorscale='Viridis'))
    
    layout=go.Layout(title_text="Original shape", title_x=0.5, title_y=0.8,title_font_size=12)
    fig = go.Figure(data=[data], layout=layout)

    fig.update_layout(title_text=title, title_x=0.5)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()

In [231]:
# initialize Diffusion map object.
mydmap = dm.DiffusionMap.from_sklearn(n_evecs = 2, alpha = 0.1, epsilon = 'bgh', k=200)

# fit to data and return the diffusion map.
dmap = mydmap.fit_transform(newX)

In [216]:
dmap.shape

(740, 2)

In [232]:
plot_2Dsub_figures(dmap)

In [218]:
# experiment with different values of alpha
alpha_list = np.linspace(0.1,1,9)
nrow = 3
ncol = 3

# plot results
subplot_titles=[f'α={round(a,4)}' for a in alpha_list]
fig = make_subplots(rows=nrow, cols=ncol,subplot_titles=subplot_titles)

a=0
for i in range(1,nrow+1):
    for j in range(1,ncol+1):
        mydmap = dm.DiffusionMap.from_sklearn(n_evecs = 2, alpha = alpha_list[a], epsilon = 'bgh', k=200)
        dmap = mydmap.fit_transform(newX)

        fig.add_trace(
            go.Scatter(x=dmap[:,0], y=dmap[:,1], mode='markers', marker=dict(
            size=3,color=dmap[:,1],opacity=0.8,colorscale='Viridis')),row=i, col=j)
        
        a+=1

fig.update_layout(height=700, width=1000, showlegend=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

In [229]:
[round(k) for k in np.linspace(5,50,9)]

[5, 11, 16, 22, 28, 33, 39, 44, 50]

In [230]:
# experiment with different values of alpha
k_list = [round(k) for k in np.linspace(5,50,9)]
nrow = 3
ncol = 3

# plot results
subplot_titles=[f'k={round(a,4)}' for a in k_list]
fig = make_subplots(rows=nrow, cols=ncol,subplot_titles=subplot_titles)

a=0
for i in range(1,nrow+1):
    for j in range(1,ncol+1):
        mydmap = dm.DiffusionMap.from_sklearn(n_evecs = 2, alpha = 0.1, epsilon = 'bgh', k=k_list[a])
        dmap = mydmap.fit_transform(newX)

        fig.add_trace(
            go.Scatter(x=dmap[:,0], y=dmap[:,1], mode='markers', marker=dict(
            size=3,color=dmap[:,1],opacity=0.8,colorscale='Viridis')),row=i, col=j)
        
        a+=1

fig.update_layout(height=700, width=1000, showlegend=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

## TMDMap

Repeat the experiment with TMDmap variant.

In [243]:
mydmap = dm.TMDmap(n_evecs = 2, alpha = 1, epsilon = 'bgh', k=200)
mydmap

In [234]:
# experiment with different values of alpha
alpha_list = np.linspace(0.1,1,9)
nrow = 3
ncol = 3

# plot results
subplot_titles=[f'α={round(a,4)}' for a in alpha_list]
fig = make_subplots(rows=nrow, cols=ncol,subplot_titles=subplot_titles)

a=0
for i in range(1,nrow+1):
    for j in range(1,ncol+1):
        mydmap = dm.TMDmap(n_evecs = 2, alpha = alpha_list[a], epsilon = 'bgh', k=200)
        dmap = mydmap.fit_transform(newX)

        fig.add_trace(
            go.Scatter(x=dmap[:,0], y=dmap[:,1], mode='markers', marker=dict(
            size=3,color=dmap[:,1],opacity=0.8,colorscale='Viridis')),row=i, col=j)
        
        a+=1

fig.update_layout(height=700, width=1000, showlegend=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

TypeError: 'NoneType' object is not callable